# Effect of U.S. Tariff Threats Map - Data Analysis & Cleaning

The project analyzed changes in U.S. imports from Canada and Mexico between November 2024 and January 2025 using transborder freight data, processed and normalized in Python for GIS mapping, and displayed in ArcGIS and Tableau.

*A quick side note that the ESRI shapefile field length is only 10 characters - that's why all of the field names are very short!*

## Importing Libraries & Data

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# open all data files in directory
jan25_data = 'data/raw/Jan2025/dot1_0125.csv'
dec24_data = 'data/raw/December2024/dot1_1224.csv'
nov24_data = 'data/raw/Nov2024/dot1_1124.csv'

# read in all data files
jan25_df = pd.read_csv(jan25_data)
dec24_df = pd.read_csv(dec24_data)
nov24_df = pd.read_csv(nov24_data)

# head of each data file
print(jan25_df.head())
print(dec24_df.head())
print(nov24_df.head())

At this point, all of our freight data is loaded into dataframes. 

## Imports

We are initially going to look at imports, because that is what the initial tariffs threatened - a "tax on imports" for Mexico and Canada.

In [ ]:
# filter for imports
jan25_df_i = jan25_df[jan25_df['TRDTYPE'] == 2]
dec24_df_i = dec24_df[dec24_df['TRDTYPE'] == 2]
nov24_df_i = nov24_df[nov24_df['TRDTYPE'] == 2]

From the .CSV documentation, we can see that the country codes for Mexico and Canada are 2010 and 1220, respectively.

In [ ]:
# filter for canada
jan25_df_ca = jan25_df_i[jan25_df_i['COUNTRY'] == 1220]
dec24_df_ca = dec24_df_i[dec24_df_i['COUNTRY'] == 1220]
nov24_df_ca = nov24_df_i[nov24_df_i['COUNTRY'] == 1220]

jan25_sum_ca = jan25_df_ca.groupby('USASTATE')['VALUE'].sum()
dec24_sum_ca = dec24_df_ca.groupby('USASTATE')['VALUE'].sum()
nov24_sum_ca = nov24_df_ca.groupby('USASTATE')['VALUE'].sum()

This is the sum of all imports from Canada to the US by state.

In [ ]:
# sum all dataframes
all_sum_cI = jan25_sum_ca + dec24_sum_ca + nov24_sum_ca

In [ ]:
# filter for mexico
jan25_df_mx = jan25_df[jan25_df['COUNTRY'] == 2010]
dec24_df_mx = dec24_df[dec24_df['COUNTRY'] == 2010]
nov24_df_mx = nov24_df[nov24_df['COUNTRY'] == 2010]

jan25_sum_mx = jan25_df_mx.groupby('USASTATE')['VALUE'].sum()
dec24_sum_mx = dec24_df_mx.groupby('USASTATE')['VALUE'].sum()
nov24_sum_mx = nov24_df_mx.groupby('USASTATE')['VALUE'].sum()

all_sum_mI = jan25_sum_mx + dec24_sum_mx + nov24_sum_mx

print(jan25_sum_mx)
print(dec24_sum_mx)
print(nov24_sum_mx)

## Exports

In [ ]:
# filter for exports
jan25_df_e = jan25_df[jan25_df['TRDTYPE'] == 1]
dec24_df_e = dec24_df[dec24_df['TRDTYPE'] == 1]
nov24_df_e = nov24_df[nov24_df['TRDTYPE'] == 1]
print(jan25_df_e.head())

In [ ]:
# sum of exports to canada for each state

# filter for canada
jan25_df_cE = jan25_df_e[jan25_df_e['COUNTRY'] == 1220]
dec24_df_cE = dec24_df_e[dec24_df_e['COUNTRY'] == 1220]
nov24_df_cE = nov24_df_e[nov24_df_e['COUNTRY'] == 1220]

jan25_sum_cE = jan25_df_cE.groupby('USASTATE')['VALUE'].sum()
dec24_sum_cE = dec24_df_cE.groupby('USASTATE')['VALUE'].sum()
nov24_sum_cE = nov24_df_cE.groupby('USASTATE')['VALUE'].sum()

all_sum_cE = jan25_sum_cE + dec24_sum_cE + nov24_sum_cE

print(jan25_sum_cE)
print(dec24_sum_cE)
print(nov24_sum_cE)


In [ ]:
# sum of exports to mexico for each state
jan25_df_mE = jan25_df_e[jan25_df_e['COUNTRY'] == 2010]
dec24_df_mE = dec24_df_e[dec24_df_e['COUNTRY'] == 2010]
nov24_df_mE = nov24_df_e[nov24_df_e['COUNTRY'] == 2010]

jan25_sum_mE = jan25_df_mE.groupby('USASTATE')['VALUE'].sum()
dec24_sum_mE = dec24_df_mE.groupby('USASTATE')['VALUE'].sum()
nov24_sum_mE = nov24_df_mE.groupby('USASTATE')['VALUE'].sum()

all_sum_mE = jan25_sum_mE + dec24_sum_mE + nov24_sum_mE

print(jan25_sum_mE)
print(dec24_sum_mE)
print(nov24_sum_mE)

## import our usa state boundaries .shp
source: [census](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)

### join the state codes to the .shp

In [ ]:
import geopandas as gpd

shp_path = r"C:\Users\test\OneDrive - University of Victoria\W2025\geog323\final-proj\data\raw\us_states\us_states.shp"

# read in shapefile
us_states = gpd.read_file(shp_path)
print(us_states.head())

In [ ]:
#Canada
# add the import data to the shapefile

# merge the november import sum data into the shapefile
us_states = us_states.merge(nov24_sum_ca, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'nov24sum_c'})
us_states['nov24sum_c'] = us_states['nov24sum_c'].fillna(0)

# merge the december import sum data into the shapefile
us_states = us_states.merge(dec24_sum_ca, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'dec24sum_c'})
us_states['dec24sum_c'] = us_states['dec24sum_c'].fillna(0)

# merge the january import sum data into the shapefile
us_states = us_states.merge(jan25_sum_ca, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'jan25sum_c'})
us_states['jan25sum_c'] = us_states['jan25sum_c'].fillna(0)

# merge the november export sum data into the shapefile
us_states = us_states.merge(nov24_sum_cE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'nov24sumcE'})

# merge the december export sum data into the shapefile
us_states = us_states.merge(dec24_sum_cE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'dec24sumcE'})

# merge the january export sum data into the shapefile
us_states = us_states.merge(jan25_sum_cE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'jan25sumcE'})

# merge the sum of canadian imports into the shapefile
us_states = us_states.merge(all_sum_cI, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'all_sum_cI'})

# merge the sum of canadian exports into the shapefile
us_states = us_states.merge(all_sum_cE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'all_sum_cE'})

# merge the deficit/surplus for canada into the shapefile
us_states = us_states.merge(all_ds_c, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'all_ds_c'})


print(us_states.head())

In [ ]:

#mexico
# merge the november import sum data into the shapefile
us_states = us_states.merge(nov24_sum_mx, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'nov24sum_m'})
us_states['nov24sum_m'] = us_states['nov24sum_m'].fillna(0) # fill NaN with 0

# merge the december import sum data into the shapefile
us_states = us_states.merge(dec24_sum_mx, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'dec24sum_m'})
us_states['dec24sum_m'] = us_states['dec24sum_m'].fillna(0) # fill NaN with 0

# merge the january import sum data into the shapefile
us_states = us_states.merge(jan25_sum_mx, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'jan25sum_m'})
us_states['jan25sum_m'] = us_states['jan25sum_m'].fillna(0) # fill NaN with 0

# merge the november export sum data into the shapefile
us_states = us_states.merge(nov24_sum_mE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'nov24summE'})

# merge the december export sum data into the shapefile
us_states = us_states.merge(dec24_sum_mE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'dec24summE'})

# merge the january export sum data into the shapefile
us_states = us_states.merge(jan25_sum_mE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'jan25summE'})

# merge the sum of all imports from mexico
us_states = us_states.merge(all_sum_mI, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'all_sum_mI'})

# merge the sum of all exports to mexico
us_states = us_states.merge(all_sum_mE, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'all_sum_mE'})

# merge the deficit/surplus for mexico
us_states = us_states.merge(all_ds_m, left_on='STUSPS', right_on='USASTATE', how='left')
us_states = us_states.rename(columns={'VALUE': 'all_ds_m'})



print(us_states.head())


In [ ]:
# create a new column that, for both Canada and Mexico, sums the difference between the november and january import sums
us_states['difn_j_ca'] = us_states['jan25sum_c'] - us_states['nov24sum_c']
us_states['difn_j_mx'] = us_states['jan25sum_m'] - us_states['nov24sum_m']

# create a new column that adds the difference sums for Canada and Mexico
us_states['difn_j_b'] = us_states['difn_j_ca'] + us_states['difn_j_mx']

print(us_states.head())

In [ ]:
# sum imports for both
us_states['all_sum_I'] = us_states['all_sum_cI'] + us_states['all_sum_mI']


## Save the file

In [ ]:

# save the shapefile
output_path = r"C:\Users\test\OneDrive - University of Victoria\W2025\geog323\final-proj\data\processed\us_states_trade.shp"
us_states.to_file(output_path)

# Attributes we have now:
# nov24sum_c, dec24sum_c, jan25sum_c, nov24sum_m, dec24sum_m, jan25sum_m, nov24sumcE, dec24sumcE, jan25sumcE, nov24summE, dec24summE, jan25summE, difn_j_ca, difn_j_mx, difn_j_b
# nov24sum_c = sum of imports from Canada in November 2024
# dec24sum_c = sum of imports from Canada in December 2024
# jan25sum_c = sum of imports from Canada in January 2025
# nov24sum_m = sum of imports from Mexico in November 2024
# dec24sum_m = sum of imports from Mexico in December 2024
# jan25sum_m = sum of imports from Mexico in January 2025
# nov24sumcE = sum of exports to Canada in November 2024
# dec24sumcE = sum of exports to Canada in December 2024
# jan25sumcE = sum of exports to Canada in January 2025
# nov24summE = sum of exports to Mexico in November 2024
# dec24summE = sum of exports to Mexico in December 2024
# jan25summE = sum of exports to Mexico in January 2025
# difn_j_ca = difference between sum of imports from Canada in January 2025 and November 2024
# difn_j_mx = difference between sum of imports from Mexico in January 2025 and November 2024
# difn_j_b = sum of difn_j_ca and difn_j_mx
# all_sum_cI = sum of imports from Canada in all three months
# all_sum_cE = sum of exports to Canada in all three months
# all_sum_mI = sum of imports from Mexico in all three months
# all_sum_mE = sum of exports to Mexico in all three months






### Sources

The project used two main data sources: North American Transborder Freight Data from the U.S. Bureau of Transportation Statistics, providing import and export shipment details from November 2024 to January 2025, and U.S. state boundary shapefiles from the United States Census Bureau, updated in 2023. The freight data, originally in CSV format, was transformed into shapefiles for mapping and analysis. Both sources were accessed in March 2025.
